In [ ]:
from IPython.display import display, Image, clear_output, HTML
import time 
import random 
import requests
from bs4 import BeautifulSoup
import json
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

In [ ]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return

def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event(timeout=10)
    text_input.disabled = True
    return event['description']

In [ ]:
def send_to_google_form(data_dict, form_url):
   
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [ ]:
def notice():

    global result
    
    print("Please read:")
    
    print("")
    
    print("we wish to record your response data")
    
    print("to an anonymised public data repository. ")
    
    print("Your data will be used for educational teaching purposes")
    
    print("practising data analysis and visualisation.")
    
    print("")
    
    print("Please click Yes if you consent to the upload.")

    

    button_yes = widgets.Button(description="Yes")
    button_no = widgets.Button(description="No")
    button_yes.on_click(register_btn_event)
    button_no.on_click(register_btn_event)
    display(button_yes,button_no)
    result=wait_for_event()
    clear_output(wait=False)

    if result['description'] == "Yes":
        
        print("Thanks - your data will be uploaded")
        result='Yes'
    
        
    elif result['description'] == "No":
        print ("No problem we hope you enjoyed the test.")
        result='No'



In [ ]:
#Picture storage
cube1= Image("cube1.PNG",width=600)
cube2= Image("cube2.PNG",width=600)
cube3= Image("cube3.PNG",width=600)
cube4= Image("cube4.PNG",width=600)
cube5= Image("cube5.PNG",width=600)
cube6= Image("cube6.PNG",width=600)
cube7= Image("cube7.PNG",width=600)
cube8= Image("cube8.PNG",width=600)
cube_list=[cube1,cube2,cube3,cube4,cube5,cube6,cube7,cube8]

In [ ]:
def validate_name():
    while True:
        name = input("Enter your four letter username.")
        if len(name) == 4 and name.isalpha():
            print("Username entered:",name)
            return name
            break
        else:
            print("Invalid name entered. Please enter a four letter name.")
            


def validate_age():
   while True:
    age = input("What is your age?")
    if age.isdigit():
        print (f"Your age is {age}.")
        return age
        break
    else:
        print("Please enter a number")



In [ ]:
def intro():

    id_instructions = """

    Enter your anonymised ID

    To generate an anonymous 4-letter unique user identifier please enter:

    - two letters based on the initials (first and last name) of a childhood friend

    - two letters based on the initials (first and last name) of a favourite actor / actress

    e.g. if your friend was called Charlie Brown and film star was Tom Cruise

    then your unique identifer would be CBTC

    """

    global name

    print(id_instructions)

    name= validate_name()

 

    global age

    age=validate_age()
   

    global gender

    button_m = widgets.Button(description="Male")
    button_f = widgets.Button(description="Female")
    button_non = widgets.Button(description="Prefer not to say")
    button_m.on_click(register_btn_event)
    button_f.on_click(register_btn_event)
    button_non.on_click(register_btn_event)

    buttons_box_gender = widgets.HBox([button_m,button_f,button_non])
    display(buttons_box_gender)
    
    gender=wait_for_event()
    clear_output(wait=False)
    gender=gender['description']
    
    welcome_message=(f"Hello, {name}, welcome to the spatial reasoning test.")
    
    print(welcome_message)

    
    time.sleep(3)
    

    instructions="You'll be given two 3D images of a shape, one of them is the same shape rotated by 180."
    
    print(instructions)
    time.sleep(4)

    

    instructions_1="Observe the shapes carefully and choose a 2D projection that cannot be created by looking at the shape presented to you. Please answer in lower case. Good luck."
    print(instructions_1)
    time.sleep(4)

    instructions_2="The 2D images are created by looking at the shape in these ways: above, under, left or right side, front or back."
    print(instructions_2)
    time.sleep(6)

    clear_output(wait=False)



In [ ]:
def test():
    
    start_time = time.time()

      
    count=0
    for i in range(0,8):
        
        
        display(cube_list[i])

        correct_answer=["d","b","c","d","b","a","a","c"]
        

        button_a = widgets.Button(description="a")
        button_b = widgets.Button(description="b")
        button_c = widgets.Button(description="c")
        button_d = widgets.Button(description="d")

        buttons_box = widgets.HBox([button_a,button_b,button_c,button_d])
        button_a.on_click(register_btn_event)
        button_b.on_click(register_btn_event)
        button_c.on_click(register_btn_event)
        button_d.on_click(register_btn_event)
        
        display(buttons_box)

        answer=wait_for_event()
        clear_output(wait=False)

    
        if answer['description']==correct_answer[i]:
            count=count+1
            print("your answer is correct")
            print(f"Your current score is {count}")
        else:
            print("Your answer is incorrect")
            print(f"Your current score is {count}")
            
            
        time.sleep(1)
        
        end_time = time.time()
    
        time_taken = end_time - start_time 
                    
        total_time = 0 
        total_time = total_time + time_taken
        total_time= round(total_time,2)

        correct_number=count


        clear_output(wait=False)
    test_result =f"You got {correct_number} correct, you spent {total_time}s on the test, thank you for participating."
    
    
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdvMjVS9U2hDx5t6vkaQ3c_2H84jfiyf4La4ayd6h9Wg_Vl9Q/viewform"


    data_dict= {
        'name' : name,
        'time_taken' : total_time,
        'score' : correct_number,
        'age' : age,
        'gender' : gender
     }
        
    if result == "Yes":
        send_to_google_form(data_dict, form_url)


    

    return test_result


In [ ]:
notice()

In [ ]:
intro()

In [ ]:
test()